In [3]:
# Import libraries we will use for this lab
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

**1. In this section we will define functions to extract the wanted data from ISA**

In [4]:
# 
url1 = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter"
url2 = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html"
r = requests.get(url1,params={'ww_i_reportmodel':133685247})
soup = bs(r.content)

# 
Xsl=soup.find_all("input",{'name':'ww_i_reportModelXsl'})
XslValue = int(Xsl[0].get("value"))

# Parameters
sections = {}
years = {}
semesters = {}
#summerwinter = {}

# Returns list of all possible key-value for the parameter
def getParameterOptions(parameterName):
    result = {}
    select = soup.find_all("select",{'name':parameterName})[0]
    options = select.find_all("option")
    for o in options:
        key = o.text
        value = o.get("value")
        if(value!='null'):
            result[key]=int(value)
    return result

sections = getParameterOptions('ww_x_UNITE_ACAD')
years = getParameterOptions('ww_x_PERIODE_ACAD')
semesters = getParameterOptions('ww_x_PERIODE_PEDAGO')
#summer_winter = getParameterOptions('ww_x_HIVERETE')

#def getParameterValues(section, year, semester, summerwinter):
def getParameterValues(section, year, semester):
    params={'ww_i_reportmodel':133685247,'ww_i_reportModelXsl':XslValue,'ww_b_list':1}
    params['ww_x_UNITE_ACAD'] = sections[section]
    params['ww_x_PERIODE_ACAD'] = years[year]
    params['ww_x_PERIODE_PEDAGO'] = semesters[semester]
    #params['ww_x_HIVERETE'] = summer_winter[summerwinter]
    return params

# We noticed that we can retrieve a page using only the GPS parameter.
#def getGPS(section, year, semester, summerwinter):
def getGPS(section, year, semester):
    #payload = getParameterValues(section, year, semester, summerwinter)
    payload = getParameterValues(section, year, semester)
    r = requests.get(url1, params = payload)
    soup = bs(r.content)
    script = soup.find_all("a")[1].get("onclick")
    code = script.replace('loadReport(\'ww_x_GPS=', '')
    code = code.replace('\');return false;','')
    return int(code)

# Returns the HTML page for the inputs parameters
#def getPage(section, year, semester, summerwinter):
def getPage(section, year, semester):
    #gps = getGPS(section, year, semester, summerwinter)
    gps = getGPS(section, year, semester)
    payload = {'ww_x_GPS':gps, 'ww_i_reportmodel':133685247, 'ww_i_reportModelXsl':XslValue}
    r = requests.get(url2, params = payload)
    return r

/Users/Yassine/anaconda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/Yassine/anaconda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


**2. In this section we will use the previous function to build a table with columns [SCIPER, GENDER, SEMESTER] that contains all semesters spent by students in Bachelor. Hence, a repeated row (n times) means that the student did the corresponding semester n times.**

In [14]:
# A function that returns a list of students in a degree(Bachelor or Master) in a range of year
def getListOfStudents(section,degree,startingYear,endingYear):
    
    # Create the resulting table
    if degree=='B':
        table= pd.DataFrame(columns=['SCIPER','GENDER','SEMESTER'])
    else:
        table= pd.DataFrame(columns=['SCIPER','GENDER','SPECIALIZATION','MINOR','SEMESTER'])
    
    # First for loop to iterate over all possible years
    for i in range(startingYear,endingYear+1):
        # Year is the string corresponding to the current year
        year=str(i)+"-"+str(i+1)
        # Calculate number of semesters
        if degree=='B':
            numberOfSemesters = 6;
        else if degree=='M':
            numberOfSemesters = 3;
        else:
            numberOfSemesters = 2;
    
        # Second for loop to iterate over all possible semesters
        for j in range(1,numberOfSemesters+1):
            # Semester is the string corresponding to the current semester
            if degree=='B':
                semester="Bachelor semestre "+str(j)
            else if degree=='M':
                semester="Master semestre "+str(j)
            else:
                if j==1:
                    semester="Projet Master automne"
                else:
                    semester="Projet Master printemps"
            # Load the page corresponding to the informatique section and the current year and semester
            request = getPage(section, year, semester)
            # Create a BeautifulSoup object for the loaded html page
            soup = bs(request.content,'html.parser')
            # Use the find all method to extract the list of all students. In this part we used the body of the html answer 
            # of the ISA server (using Postman) to adjust the index of this list so that each element of the list corresponds
            # to a student and not other data.
            list_of_students=soup.findAll('tr')[2:]
        
            # Third loop to go throw the list and extract the student's information
            for student in list_of_students:
                # Get the list of all the attributes of a current student (again, the positions of the attributes found with Postman)
                current_student_infos=student.findAll('td')
                # Extract the gender attribute
                current_student_gender=current_student_infos[0].string
                # Extract the sciper attribute
                current_student_sciper=current_student_infos[10].string
                if 
                # Build a new row (dataFrame) with the extracted information
                current_student_new_table_entry = pd.DataFrame([[current_student_sciper,current_student_gender,str(j)]],columns=['SCIPER','GENDER','SEMESTER'])
                # Append the new row to the resulting table
                table=table.append(current_student_new_table_entry)

    # Once the table is populated change 'Monsieur' with 'M' and 'Madame' with 'F' in thegender column
    table=table.replace({'GENDER': {'Monsieur': 'M','Madame':'F'}})
    #index the resulting table
    table.index=range(0,len(table))
    return table

SyntaxError: invalid syntax (<ipython-input-14-dbfb10442f20>, line 17)

In [13]:
# Specify the section, level and years for exercise 1
section="Informatique"
degree='B'
startingYear=2007
endingYear=2016
sciper_gender_table = getListOfStudents(section,degree,startingYear,endingYear)
sciper_gender_table.head()

/Users/Yassine/anaconda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/Yassine/anaconda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


,SCIPER,GENDER,SEMESTER
0,169569,M,1
1,174905,M,1
2,173922,M,1
3,179406,M,1
4,179428,M,1


**3. In this part we will compute the set of students that did their semester 1 and 6 between 2007 and 2007 
and other sets that we will not use later but just to have an idea of the bachelor cycle of different students of IN**

In [ ]:
#Compute the set of all scipers
all_students_distinct= set(sciper_gender_table.SCIPER)

#Compute the set of all scipers corresponding to students that did their semester 1 between 2007 and 2017
students_with_semester_1_entry=set(sciper_gender_table.loc[sciper_gender_table['SEMESTER']=='1','SCIPER'])

#Compute the set of all scipers corresponding to students that did their semester 6 between 2007 and 2017
students_with_semester_6_entry=set(sciper_gender_table.loc[sciper_gender_table['SEMESTER']=='6','SCIPER'])

#Compute the set of all scipers corresponding to students that did their semester 1 and 6 between 2007 and 2017
#THIS IS THE SET WE ARE INTERESTED IN FOR THE TEST
students_with_semester_1_and_6_entry=students_with_semester_1_entry.intersection(students_with_semester_6_entry)

#Compute the set of all scipers corresponding to students did their semester 6 between 2007 and 2017 but did semester 1 before 2007
#by substracting students_with_semester_1_and_6_entry from student_with_semester_6
#we nedd to copy students_with_semester_6_entry because the method difference_update that substract doesn't make a copy
#and change directly the calling set
student_with_semester_6_and_not_1=students_with_semester_6_entry.copy()
student_with_semester_6_and_not_1.difference_update(students_with_semester_1_and_6_entry)


print("The total number of students is: "+ str(len(all_students_distinct)))
print("The total number of students that did their semester 1 between 2007 and 2017 is: "+str(len(students_with_semester_1_entry)))
print("The total number of students that did their semester 6 between 2007 and 2017 is: "+str(len(students_with_semester_6_entry)))
print("The total number of students that did their semester 1 and 6 between 2007 and 2017 is: "+ str (len(students_with_semester_1_and_6_entry)))
print("The total number of students that did their semester 6 between 2007 and 2017 but did semester 1 before 2007 is: "+ str (len(student_with_semester_6_and_not_1)))

**Coclusion:** The number of students that reached their sixth semester between spring 2007 and spring 2017 are 516 students. But only 379 of them began their firts semester of bachelor in or after autaumn 2007. Students who reached the sixth semester between 2007 and 2017 but their beginning year is unkown from the data are not taken into account because the duration of their bachelor is unkown.**
**------------------------------------------------------------------------------------------------------------------**

**4. In this part we will compute the table showing the total number of month spent in the bachelor relative to the students we found in the previous section (students that did their semester 1 and 6 between 2007 and 2017)**

One way to do that is to merge a dataFrame containing one column 'SCIPER' containing all scipers of students that did their semester 1 and 6 between 2007 and 2017 and the original table 'sciper_gender_table' with an **inner join** which will result in all rows in 'sciper_gender_table' that have a scipr numbers we are interested in.

In [ ]:
#List of students that did their semester 1 and 6 between 2007 and 2017 as a dataFrame
students_with_semester_1_and_6_entry_DFrame=pd.DataFrame(list(students_with_semester_1_and_6_entry),columns=['SCIPER'])
students_with_semester_1_and_6_entry_DFrame.head()

In [ ]:
# merge the previous dataFrame with the original table
sciper_gender_table_semester_1_and_6=pd.merge(sciper_gender_table,students_with_semester_1_and_6_entry_DFrame,how='inner')
sciper_gender_table_semester_1_and_6.head(10)

**Conclusion:** The resulting table consist of multiple raws for each SCIPER. The number of raws with the same SCIPER represents the number of semesters spent by the student with this SCIPER in Bachelor. 
Hence, we are assuming that a semester during which **a student is not registred in any course doesn't count in the total number of semesters a student spend in Bachelor.** This assumption avoid us to count extra semesters where students are not registred so are not taking any course. Hence, we only count semesters in which the student is effectively regestred and taking at least ine course.

**4. In this section we will group the resulting table by student to compute the number of semesters spent in the bachelor for each students and then transform the number of semesters to the number of months**

In [ ]:
# Group by SCIPER and GENDER to get the table of all different students
# The count method will count the number of raws corresponding to each SCIPER in the SMESTER column
# Which corresponds as we said earlier to the number of semester spent by the student doing his bachelor
total_number_of_semesters_of_bachelor=sciper_gender_table_semester_1_and_6.groupby(by=['SCIPER','GENDER']).count()

# Change the name of SEMESTER column to 'Number of semesters of bachelor'
total_number_of_semesters_of_bachelor.columns=['NUMBER OF SEMESTERS OF BACHELOR']

total_number_of_semesters_of_bachelor.head()

In [221]:
# Create the table that shows the number of months of bachelor per student as a copy from the table of semesters
total_number_of_months_of_bachelor=total_number_of_semesters_of_bachelor.copy()

# Add a new column that represents the number of months and fill it by multiplyng the correspondent number of
#semesters by 6 (1 semester= 6 months)
total_number_of_months_of_bachelor['NUMBER OF MONTHS OF BACHELOR']= 6* total_number_of_months_of_bachelor['NUMBER OF SEMESTERS OF BACHELOR']

# Remove the column corresponding to the number of semesters and sort according to the number of months
total_number_of_months_of_bachelor=total_number_of_months_of_bachelor[['NUMBER OF MONTHS OF BACHELOR']].sort_values('NUMBER OF MONTHS OF BACHELOR')

total_number_of_months_of_bachelor.head()


,,NUMBER OF MONTHS OF BACHELOR
SCIPER,GENDER,
202543,M,36
205844,M,36
205709,M,36
205677,M,36
205675,M,36


**Problem**: We can see from the table that the first entry corresponds to a student that finished his bachelor in 24 months while the minimum is 36. The data corresponding to this student is missing for semester 3 and 4 are missing as shown above:

In [222]:
# Show all rows with sciper='204222'
sciper_gender_table[sciper_gender_table['SCIPER']=='204222']

,SCIPER,GENDER,SEMESTER


Hence, removing this row will make our data more robust:

In [223]:
total_number_of_months_of_bachelor=total_number_of_months_of_bachelor[1:]
total_number_of_months_of_bachelor.head()

,,NUMBER OF MONTHS OF BACHELOR
SCIPER,GENDER,
205844,M,36
205709,M,36
205677,M,36
205675,M,36
205662,M,36


Now we can group our table according to the gender we can compute for example the mean number of months spent in the bachelor for each gender:

In [224]:
#reindex the table of total number of months because the gender column was an index and then group by GENDER
#and compute the mean for each gender and take the 'Number of months of bachelor' column
mean_number_of_months_of_bachelor_by_gender=total_number_of_months_of_bachelor.reset_index().groupby(by=['GENDER']).mean()[['NUMBER OF MONTHS OF BACHELOR']]
##Rename the column of the mean number of months spent in the Bachelor
mean_number_of_months_of_bachelor_by_gender.columns=['Mean number of months spent in the Bachelor']
mean_number_of_months_of_bachelor_by_gender

,Mean number of months spent in the Bachelor
GENDER,
F,44.400000
M,42.417722


**In this section we will apply a test to the data to know whether the difference of the period spent to finish the Bachelor between man and women is statistically significant:**

#TEST

In [15]:
# Specify the section, level and years for exercise 1
section="Informatique"
degree='M'
startingYear=2007
endingYear=2016
sciper_gender_table = getListOfStudents(section,degree,startingYear,endingYear)
sciper_gender_table.head()

/Users/Yassine/anaconda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/Yassine/anaconda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


,GENDER,MINOR,SCIPER,SEMESTER,SPECIALIZATION
0,M,NaN,153066,1,NaN
1,F,NaN,180027,1,NaN
2,M,NaN,152232,1,NaN
3,M,NaN,177395,1,NaN
4,M,NaN,161970,1,NaN


In [16]:
sciper_gender_table

,GENDER,MINOR,SCIPER,SEMESTER,SPECIALIZATION
0,M,NaN,153066,1,NaN
1,F,NaN,180027,1,NaN
2,M,NaN,152232,1,NaN
3,M,NaN,177395,1,NaN
4,M,NaN,161970,1,NaN
5,M,NaN,166258,1,NaN
6,M,NaN,173600,1,NaN
7,M,NaN,178879,1,NaN
8,F,NaN,154573,1,NaN
9,M,NaN,160492,1,NaN
